In [31]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder 
from catboost import CatBoostClassifier
import re
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aigozhiev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aigozhiev\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [11]:
df = pd.read_csv('../data/twitter_training.csv')
df.columns = ['id','game','target','text']
df = df.drop('id', axis=1)

In [57]:
df["text"] = df['text'].str.lower()
df["text"] = df['text'].apply(lambda x: str(x))
df["text"] = df.text.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', '', x))
df['text'] = df.text.apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [58]:
stopwords_nltk = nltk.corpus.stopwords
stop_words = stopwords_nltk.words('english')

['i', 'me', 'my', 'myself', 'we']

In [66]:
y = df['target']
x = df['text']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [69]:
pipe = Pipeline([
    ('bow', CountVectorizer(tokenizer=word_tokenize,
                stop_words=stop_words,
                ngram_range=(1,2))),
    ('tf_idf', TfidfTransformer()),
    ('clf', LogisticRegression(max_iter=200, random_state=42))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\aigozhiev\AppData\Local\miniconda3\envs\mvenv\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\aigozhiev\AppData\Local\miniconda3\envs\mvenv\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(
C:\Users\aigozhiev\AppData\Local\miniconda3\envs\mvenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver 

0.8706567583852179

In [82]:
'+'.join(pipe.named_steps.keys())

'bow+tf_idf+clf'